In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import numpy as np
import scipy
import os
import scalebar
import nept

from loading_data import get_data
from analyze_tuning_curves import get_only_tuning_curves
from utils_plotting import plot_over_space
from utils_maze import get_zones, get_bin_centers, get_matched_trials

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "trials", "decoding", "shuffled")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [ ]:
import info.r063d2 as r063d2
import info.r063d3 as r063d3
infos = [r063d2, r063d3]
from run import analysis_infos
# infos = analysis_infos

In [ ]:
def bin_spikes(spikes, time, dt, window=None, gaussian_std=None, normalized=True):
    """Bins spikes using a sliding window.

    Parameters
    ----------
    spikes: list
        Of nept.SpikeTrain
    time: np.array
    window: float or None
        Length of the sliding window, in seconds. If None, will default to dt.
    dt: float
    gaussian_std: float or None
    normalized: boolean

    Returns
    -------
    binned_spikes: nept.AnalogSignal

    """
    if window is None:
        window = dt

    bin_edges = time

    given_n_bins = window / dt
    n_bins = int(round(given_n_bins))
    if abs(n_bins - given_n_bins) > 0.01:
        warnings.warn("dt does not divide window evenly. "
                      "Using window %g instead." % (n_bins*dt))

    if normalized:
        square_filter = np.ones(n_bins) * (1 / n_bins)
    else:
        square_filter = np.ones(n_bins)

    counts = np.zeros((len(spikes), len(bin_edges) - 1))
    for idx, spiketrain in enumerate(spikes):
        counts[idx] = np.convolve(np.histogram(spiketrain.time, bins=bin_edges)[0].astype(float),
                                  square_filter, mode="same")

    if gaussian_std is not None:
        counts = nept.gaussian_filter(counts, gaussian_std, dt=dt, normalized=normalized, axis=1)

    return nept.AnalogSignal(counts, bin_edges[:-1])

In [ ]:
def plot_summary_individual(info, likelihood_true, likelihood_shuffs, position, lfp, spikes, start, stop, 
                            zones, maze_segments, colours, filepath=None, savefig=False):
    buffer=0.1
    
    means = [np.nansum(likelihood[zones[trajectory]]) for trajectory in maze_segments]
    
#     combined_shuff = {trajectory: [] for trajectory in maze_segments}
#     for likelihood_shuff in likelihood_shuffs:
#         for trajectory in maze_segments:
#             combined_shuff[trajectory].append(np.nansum(likelihood_shuffs[zones[trajectory]], axis=0))
    means_shuff = [np.nansum(likelihood_shuff[zones[trajectory]], axis=0) for trajectory in maze_segments]
    sems_shuff = [scipy.stats.sem(likelihood_shuff[zones[trajectory]], axis=0, nan_policy="omit") for trajectory in maze_segments]

    sliced_spikes = [spiketrain.time_slice(start-buffer, stop+buffer) for spiketrain in spikes]

    rows = len(sliced_spikes)
    add_rows = int(rows / 8)

    ms = 600 / rows
    mew = 0.7
    spike_loc = 1

    fig = plt.figure(figsize=(8, 8))
    gs1 = gridspec.GridSpec(3, 2)
    gs1.update(wspace=0.3, hspace=0.3)

    ax1 = plt.subplot(gs1[1:, 0])
    for idx, neuron_spikes in enumerate(sliced_spikes):
        ax1.plot(neuron_spikes.time, np.ones(len(neuron_spikes.time)) + (idx * spike_loc), '|',
                 color='k', ms=ms, mew=mew)
    ax1.axis('off')

    ax2 = plt.subplot(gs1[0, 0], sharex=ax1)

    swr_highlight = "#fc4e2a"
    start_idx = nept.find_nearest_idx(lfp.time, start - buffer)
    stop_idx = nept.find_nearest_idx(lfp.time, stop + buffer)
    ax2.plot(lfp.time[start_idx:stop_idx], lfp.data[start_idx:stop_idx], color="k", lw=0.3, alpha=0.9)

    start_idx = nept.find_nearest_idx(lfp.time, start)
    stop_idx = nept.find_nearest_idx(lfp.time, stop)
    ax2.plot(lfp.time[start_idx:stop_idx], lfp.data[start_idx:stop_idx], color=swr_highlight, lw=0.6)
    ax2.axis("off")
    
    ax1.axvline(lfp.time[start_idx], linewidth=1, color=swr_highlight)
    ax1.axvline(lfp.time[stop_idx], linewidth=1, color=swr_highlight)
    ax1.axvspan(lfp.time[start_idx], lfp.time[stop_idx], alpha=0.5, color=swr_highlight)

    scalebar.add_scalebar(ax2, matchy=False, bbox_transform=fig.transFigure,
                          bbox_to_anchor=(0.25, 0.05), units='ms')

    likelihood[np.isnan(likelihood)] = 0
    
    xx, yy = np.meshgrid(info.xedges, info.yedges)
    xcenters, ycenters = get_bin_centers(info)
    xxx, yyy = np.meshgrid(xcenters, ycenters)

    maze_highlight = "#fed976"
    ax3 = plt.subplot(gs1[0, 1])
    sliced_position = position.time_slice(info.task_times["phase3"].starts, info.task_times["phase3"].stops)
    ax3.plot(sliced_position.x, sliced_position.y, ".", color=maze_highlight, ms=1, alpha=0.2)
    pp = ax3.pcolormesh(xx, yy, likelihood_true, cmap='bone_r')
    ax3.contour(xxx, yyy, zones["u"], levels=0, colors=colours["u"])
    ax3.contour(xxx, yyy, zones["shortcut"], levels=0, colors=colours["shortcut"])
    ax3.contour(xxx, yyy, zones["novel"], levels=0, colors=colours["novel"])
    plt.colorbar(pp)
    ax3.axis('off')

    ax4 = plt.subplot(gs1[1:2, 1])
    n = np.arange(len(maze_segments))
    ax4.bar(n, means, 
            color=[colours["u"], colours["shortcut"], colours["novel"], colours["other"]], edgecolor='k')
    ax4.set_xticks(n)
    ax4.set_xticklabels([], rotation=90)
    ax4.set_ylim([0, 1.])
    ax4.set_title("True proportion", fontsize=14)
    
    ax5 = plt.subplot(gs1[2:, 1], sharey=ax4)
    n = np.arange(len(maze_segments))
    ax5.bar(n, np.nanmean(means_shuff, axis=0), 
            yerr=np.nanmean(sems_shuff, axis=0), 
            color=[colours["u"], colours["shortcut"], colours["novel"], colours["other"]], edgecolor='k')
    ax5.set_xticks(n)
    ax5.set_xticklabels(maze_segments, rotation=90)
    ax5.set_ylim([0, 1.])
    ax5.set_title("Shuffled proportion", fontsize=14)
    
    plt.tight_layout()
    
    if savefig:
        plt.savefig(filepath)
        plt.close()
    else:
        plt.show()

In [ ]:
tuning_curves = tuning_curves_fromdata
tuning_curves = tuning_curves.reshape(tc_shape[0], tc_shape[1] * tc_shape[2])
start = phase_swrs[task_time].starts[0]
stop = phase_swrs[task_time].stops[0]

likelihood = get_likelihood(spikes, tuning_curves, tc_shape, start, stop)

likelihood_shuffs = []

for _ in range(3):
    tuning_curves_shuff = np.random.permutation(tuning_curves_fromdata)
    tuning_curves_shuff = tuning_curves_shuff.reshape(tc_shape[0], tc_shape[1] * tc_shape[2])

    likelihood_shuff = get_likelihood(spikes, tuning_curves_shuff, tc_shape, start, stop)
    
    likelihood_shuffs.append(likelihood_shuff)

filename = info.session_id + "_" + task_time + "_summary-swr" + str(i) + "_sum.png"
filepath = os.path.join(output_filepath, "swr", filename)
plot_summary_individual(info, likelihood, likelihood_shuffs, position, lfp, spikes, start, stop,
                        zones, maze_segments, colours, filepath, savefig=False)

In [ ]:
def plot_likelihood_overspace(info, position, likelihoods, zones, colours, filepath=None):
    
    xx, yy = np.meshgrid(info.xedges, info.yedges)
    xcenters, ycenters = get_bin_centers(info)
    xxx, yyy = np.meshgrid(xcenters, ycenters)
    
    sliced_position = position.time_slice(info.task_times["phase3"].starts, info.task_times["phase3"].stops)
    plt.plot(sliced_position.x, sliced_position.y, "b.", ms=1, alpha=0.2)
    pp = plt.pcolormesh(xx, yy, np.nanmean(likelihoods, axis=0), vmax=0.2, cmap='bone_r')
    plt.contour(xxx, yyy, zones["u"], levels=0, colors=colours["u"], corner_mask=False)
    plt.contour(xxx, yyy, zones["shortcut"], levels=0, colors=colours["shortcut"], corner_mask=False)
    plt.contour(xxx, yyy, zones["novel"], levels=0, colors=colours["novel"], corner_mask=False)

    plt.colorbar(pp)
    plt.axis('off')
    if filepath is not None:
        plt.savefig(os.path.join(output_filepath, filename+".png"))
        plt.close()
    else:
        plt.show()

In [ ]:
def get_likelihood(spikes, tuning_curves, tc_shape, start, stop):
    sliced_spikes = [spiketrain.time_slice(start, stop) for spiketrain in spikes]
    t_window = stop-start # 0.1 for running, 0.025 for swr
    counts = bin_spikes(sliced_spikes, np.array([start, stop]), dt=t_window, window=t_window,
                        gaussian_std=0.0075, normalized=False)
    likelihood = nept.bayesian_prob(counts, tuning_curves, binsize=t_window, min_neurons=3, min_spikes=1)

    return likelihood.reshape(tc_shape[1], tc_shape[2])

In [ ]:
def plot_combined(summary_likelihoods, n_all_swrs, task_times, maze_segments, n_sessions, colours, filename=None):
    
    trajectory_means = {key: [] for key in maze_segments}
    trajectory_sems = {key: [] for key in maze_segments}
    max_val = 0
    
    for trajectory in maze_segments:
        for task_time in task_times:
            if len(summary_likelihoods[task_time][trajectory]) > 0:
                max_val = max(np.max(summary_likelihoods[task_time][trajectory]), max_val)

        trajectory_means[trajectory] = [np.nanmean(summary_likelihoods[task_time][trajectory]) for task_time in task_times]
        trajectory_sems[trajectory] = [scipy.stats.sem(summary_likelihoods[task_time][trajectory]) for task_time in task_times] 

    fig = plt.figure(figsize=(12,6))
    gs1 = gridspec.GridSpec(1, 4)
    gs1.update(wspace=0.3, hspace=0.)

    n = np.arange(len(task_times))
    ax1 = plt.subplot(gs1[0])
    ax1.bar(n, trajectory_means["u"], yerr=trajectory_sems["u"], color=colours["u"])
    ax2 = plt.subplot(gs1[1])
    ax2.bar(n, trajectory_means["shortcut"], yerr=trajectory_sems["shortcut"], color=colours["shortcut"])
    ax3 = plt.subplot(gs1[2])
    ax3.bar(n, trajectory_means["novel"], yerr=trajectory_sems["novel"], color=colours["novel"])
    ax4 = plt.subplot(gs1[3])
    ax4.bar(n, trajectory_means["other"], yerr=trajectory_sems["other"], color=colours["other"])
    
    for ax in [ax1, ax2, ax3, ax4]:
        ax.set_ylim([0, max_val])
        
        ax.set_xticks(np.arange(len(task_times)))
        ax.set_xticklabels(task_times, rotation = 90)

        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for i, task_time in enumerate(task_times):
            ax.text(i, 0.01, str(n_all_swrs[task_time]), ha="center", fontsize=14)

    for ax in [ax2, ax3, ax4]:
        ax.set_yticklabels([])
        
    plt.text(1., max_val, "n sessions: "+ str(n_sessions), horizontalalignment='left', 
             verticalalignment='top', fontsize=14)

    fig.suptitle(filename, fontsize=18)
#     ax1.set_ylabel("Proportion")

    legend_elements = [Patch(facecolor=colours["u"], edgecolor='k', label="u"),
                       Patch(facecolor=colours["shortcut"], edgecolor='k', label="shortcut"),
                       Patch(facecolor=colours["novel"], edgecolor='k', label="novel"),
                       Patch(facecolor=colours["other"], edgecolor='k', label="other")]
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1.0))
    
    gs1.tight_layout(fig)
    
    if filename is not None:
        plt.savefig(os.path.join(output_filepath, filename+".png"))
        plt.close()
    else:
        plt.show()

In [ ]:
def plot_stacked_summary(summary_likelihoods, n_all_swrs, task_times, maze_segments, n_sessions, colours, filename=None):
    
    trajectory_means = {key: [] for key in maze_segments}
    trajectory_sems = {key: [] for key in maze_segments}

    for trajectory in maze_segments:              
        trajectory_means[trajectory] = [np.nanmean(summary_likelihoods[task_time][trajectory]) for task_time in task_times]
        trajectory_sems[trajectory] = [scipy.stats.sem(summary_likelihoods[task_time][trajectory]) for task_time in task_times] 

    fig, ax = plt.subplots(figsize=(7,5))
    n = np.arange(len(task_times))
    pu = plt.bar(n, trajectory_means["u"], yerr=trajectory_sems["u"], color=colours["u"])
    ps = plt.bar(n, trajectory_means["shortcut"], yerr=trajectory_sems["shortcut"],
                 bottom=trajectory_means["u"], color=colours["shortcut"])
    pn = plt.bar(n, trajectory_means["novel"], yerr=trajectory_sems["novel"],
                 bottom=np.array(trajectory_means["u"])+np.array(trajectory_means["shortcut"]), color=colours["novel"])
    po = plt.bar(n, trajectory_means["other"], yerr=trajectory_sems["other"],
                 bottom=np.array(trajectory_means["u"])+np.array(trajectory_means["shortcut"])+np.array(trajectory_means["novel"]), 
                 color=colours["other"])
    plt.xticks(n, task_times)
    plt.title(filename)

    for i, task_time in enumerate(task_times):
        ax.text(i, 0.01, str(n_all_swrs[task_time]), ha="center", fontsize=14)
        
    plt.text(2.8, -0.15, "n sessions: "+ str(n_sessions), fontsize=14)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    plt.tight_layout()
    if filename is not None:
        plt.savefig(os.path.join(output_filepath, filename+".png"))
        plt.close()
    else:
        plt.show()

In [ ]:
def compute_likelihoods(tuning_curves_fromdata, tc_shape, spikes, phase_swrs, shuffled_id=False):
    
    if shuffled_id:
        tuning_curves = np.random.permutation(tuning_curves_fromdata)
    else:
        tuning_curves = tuning_curves_fromdata

    tuning_curves = tuning_curves.reshape(tc_shape[0], tc_shape[1] * tc_shape[2])

    likelihoods_sum = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    raw_likelihoods = {task_time: [] for task_time in task_times}

    for i, task_time in enumerate(task_times):
        for (start, stop) in zip(phase_swrs[task_time].starts, phase_swrs[task_time].stops):
            likelihood = get_likelihood(spikes, tuning_curves, tc_shape, start, stop)
            raw_likelihoods[task_time].append(likelihood)

            for trajectory in maze_segments:
                likelihoods_sum[task_time][trajectory].append(np.nansum(likelihood[zones[trajectory]]))

    return likelihoods_sum, raw_likelihoods

In [ ]:
# set the random seed so results are consistent between runs
np.random.seed(0)
n_shuffles = 3
percentile_thresh = 80

colours = dict()
colours["u"] = "#2b8cbe"
colours["shortcut"] = "#31a354"
colours["novel"] = "#d95f0e"
colours["other"] = "#bdbdbd"

# swr params
z_thresh = 2.0
power_thresh = 3.0
merge_thresh = 0.02
min_length = 0.05
swr_thresh = (140.0, 250.0)

task_times = ["prerecord", "pauseA", "pauseB", "postrecord"]
maze_segments = ["u", "shortcut", "novel", "other"]

n_sessions = len(infos)
# likelihoods_sum = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
all_likelihoods_true = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
all_likelihoods_shuff = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
all_likelihoods_proportion = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
all_likelihoods_true_passthresh = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}

n_all_swrs = {task_time: 0 for task_time in task_times}
    
for info in infos:
    print(info.session_id)
    events, position, spikes, lfp, _ = get_data(info)
    
    tuning_curves_fromdata = get_only_tuning_curves(info,
                                           position,
                                           spikes,
                                           info.task_times["phase3"])
    
    tc_shape = tuning_curves_fromdata.shape
    
    # Define zones
    zones = dict()
    zones["u"], zones["shortcut"], zones["novel"] = get_zones(info, position, subset=True)
    combined_zones = zones["u"] + zones["shortcut"] + zones["novel"]
    zones["other"] = ~combined_zones

    # Find SWRs for the whole session
    
    swrs = nept.detect_swr_hilbert(lfp, fs=info.fs, thresh=swr_thresh, z_thresh=z_thresh,
                                   power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)
    swrs = nept.find_multi_in_epochs(spikes, swrs, min_involved=4)

    rest_epochs = nept.rest_threshold(position, thresh=12., t_smooth=0.8)

    # Restrict SWRs to those during epochs of interest during rest    
    phase_swrs = dict()
    n_swrs = {task_time: 0 for task_time in task_times}
    
    for task_time in task_times:
        epochs_of_interest = info.task_times[task_time].intersect(rest_epochs)

        phase_swrs[task_time] = epochs_of_interest.overlaps(swrs)
        phase_swrs[task_time] = phase_swrs[task_time][phase_swrs[task_time].durations >= 0.05]
        
        n_swrs[task_time] += phase_swrs[task_time].n_epochs
        n_all_swrs[task_time] += phase_swrs[task_time].n_epochs
    
    session_likelihoods_true, raw_likelihoods_true = compute_likelihoods(tuning_curves_fromdata, 
                                                   tc_shape, 
                                                   spikes, 
                                                   phase_swrs, 
                                                   shuffled_id=False)
 
    combined_likelihoods_shuff = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    raw_likelihoods_shuffs = {task_time: [] for task_time in task_times}
    
    for i_shuffle in range(n_shuffles):
        session_likelihoods_shuff, raw_likelihoods_shuff = compute_likelihoods(tuning_curves_fromdata,
                                                        tc_shape,
                                                        spikes,
                                                        phase_swrs,
                                                        shuffled_id=True)
        # TODO: reshape raw_likelihoods_shuffs???
        for task_time in task_times:
            raw_likelihoods_shuffs[task_time].append(raw_likelihoods_shuff[task_time])
            for trajectory in maze_segments:
                combined_likelihoods_shuff[task_time][trajectory].append(np.array(session_likelihoods_shuff[task_time][trajectory]))
    

    compareshuffle = {task_time: {trajectory: 0 for trajectory in maze_segments} for task_time in task_times}
    percentiles = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    passedshuffthresh = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    
    keep_idx = {task_time: [] for task_time in task_times}
    
    for task_time in task_times:
        for trajectory in maze_segments:
            for idx, event in enumerate(range(len(session_likelihoods_true[task_time][trajectory]))):
                percentile = scipy.stats.percentileofscore(np.sort(np.array(combined_likelihoods_shuff[task_time][trajectory])[:,event]), 
                                                           session_likelihoods_true[task_time][trajectory][event])
                percentiles[task_time][trajectory].append(percentile)
                if percentile >= percentile_thresh:
                    compareshuffle[task_time][trajectory] += 1
                    keep_idx[task_time].append(idx)

    morelikelythanshuffle_proportion = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}
    mean_combined_likelihoods_shuff = {task_time: {trajectory: [] for trajectory in maze_segments} for task_time in task_times}

    for task_time in task_times:
        for trajectory in maze_segments:
            passedshuffthresh[task_time][trajectory] = np.array(session_likelihoods_true[task_time][trajectory])[np.sort(np.unique(keep_idx[task_time]))]
            
            morelikelythanshuffle_proportion[task_time][trajectory].append(compareshuffle[task_time][trajectory] / len(session_likelihoods_true[task_time][trajectory]))
            mean_combined_likelihoods_shuff[task_time][trajectory] = np.nanmean(combined_likelihoods_shuff[task_time][trajectory], axis=0)
            
            all_likelihoods_true[task_time][trajectory].extend(session_likelihoods_true[task_time][trajectory])
            all_likelihoods_true_passthresh[task_time][trajectory].extend(passedshuffthresh[task_time][trajectory])
            all_likelihoods_shuff[task_time][trajectory].extend(mean_combined_likelihoods_shuff[task_time][trajectory])
            all_likelihoods_proportion[task_time][trajectory].extend(morelikelythanshuffle_proportion[task_time][trajectory])
            
            # plot percentiles
            fig, ax = plt.subplots()
            n = np.arange(len(percentiles[task_time][trajectory]))
            plt.bar(n, np.sort(percentiles[task_time][trajectory]), color=colours[trajectory])
            ax.axhline(percentile_thresh, ls="--", lw=1.5, color="k")
            title = info.session_id + " individual SWR percentile with shuffle" + str(n_shuffles) + " for " + task_time + " " + trajectory
            plt.title(title, fontsize=11)
            plt.tight_layout()
            plt.savefig(os.path.join(output_filepath, "percentiles", title))
            plt.close()
            
        filename = info.session_id+"-average-likelihood-overspace_"+task_time
        if len(session_likelihoods_true[task_time]) > 0:
            plot_likelihood_overspace(info, position, raw_likelihoods_true[task_time], zones, colours, filename)
            
        filename = info.session_id+"-average-likelihood-overspace_"+task_time+"-shuffled"
        if len(session_likelihoods_true[task_time]) > 0:
            plot_likelihood_overspace(info, position, raw_likelihoods_shuff[task_time], zones, colours, filename)
            
    filename = info.session_id + " proportion of SWRs above "+str(percentile_thresh)+" percentile"
    plot_combined(morelikelythanshuffle_proportion, n_swrs, task_times, maze_segments, n_sessions=1, colours=colours, filename=filename)

    filename = info.session_id + " average posteriors during SWRs_sum-shuffled"+str(n_shuffles)
    plot_combined(mean_combined_likelihoods_shuff, n_swrs, task_times, maze_segments, n_sessions=1, colours=colours, filename=filename)

    filename = info.session_id + " average posteriors during SWRs_sum-true"
    plot_combined(session_likelihoods_true, n_swrs, task_times, maze_segments, n_sessions=1, colours=colours, filename=filename)
    
    filename = info.session_id + " average posteriors during SWRs_sum-true_passthresh"
    plot_combined(passedshuffthresh, n_swrs, task_times, maze_segments, n_sessions=1, colours=colours, filename=filename)
    
    for task_time in task_times:
        for i, (raw_true, raw_shuffs) in enumerate(zip(raw_likelihoods_true[task_time], raw_likelihoods_shuffs[task_time])):
            filename = info.session_id + "_" + task_time + "_summary-swr" + str(i) + ".png"
            filepath = os.path.join(output_filepath, "swr", filename)
            plot_summary_individual(info, raw_true, raw_shuffs, position, lfp, spikes, start, stop,
                                    zones, maze_segments, colours, filepath, savefig=True)
    
filename = "Average posteriors during SWRs_sum-shuffled"+str(n_shuffles)
plot_combined(all_likelihoods_shuff, n_all_swrs, task_times, maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

filename = "Average posteriors during SWRs_sum-true"
plot_combined(all_likelihoods_true, n_all_swrs, task_times, maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

filename = "Average posteriors during SWRs_sum-true_passthresh"
plot_combined(all_likelihoods_true_passthresh, n_all_swrs, task_times, maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

filename = "Average posteriors during SWRs_sum-stacked-shuffled"+str(n_shuffles)
plot_stacked_summary(all_likelihoods_shuff, n_all_swrs, task_times, maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

filename = "Average posteriors during SWRs_sum-stacked-true"
plot_stacked_summary(all_likelihoods_true, n_all_swrs, task_times, maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

filename = "Average posteriors during SWRs_sum-stacked-true_passthresh"
plot_stacked_summary(all_likelihoods_true_passthresh, n_all_swrs, task_times, maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

filename = "Proportion of SWRs above the "+str(percentile_thresh)+" percentile (shuffle" + str(n_shuffles) + ")"
plot_combined(all_likelihoods_proportion, n_swrs, task_times, maze_segments, n_sessions=len(infos), colours=colours, filename=filename)

In [ ]:
len(raw_likelihoods_true[task_time])

In [ ]:
len(np.array(raw_likelihoods_shuffs[task_time][0]))

In [ ]:
len(raw_likelihoods_shuffs[task_time][0])

In [ ]:
len(combined_shuff[task_time])

In [ ]:
raw_likelihoods_true[task_time]

In [ ]:
for task_time in task_times:
    for idx in range(phase_swrs[task_time].n_epochs):
        filename = info.session_id + "_" + task_time + "_summary-swr" + str(i) + ".png"
        filepath = os.path.join(output_filepath, "swr", filename)
        plot_summary_individual(info, raw_likelihoods_true[task_time][idx], 
                                raw_likelihoods_shuffs[task_time][idx], 
                                position, lfp, spikes, 
                                phase_swrs[task_time].starts[idx], 
                                phase_swrs[task_time].stops[idx],
                                zones, maze_segments, colours, filepath, savefig=False)

In [ ]:
len(raw_likelihoods_shuffs["pauseA"][1])

In [ ]:
len(raw_likelihoods_true["pauseA"])

In [ ]:
1/0
with open(pickled_path, 'wb') as fileobj:
    pickle.dump(output, fileobj)